# 强化学习+深度学习 实现关卡平衡

## 下载实验代码

In [ ]:
%%bash
mkdir -p ~/SageMaker/frozen-lake
curl https://s3.amazonaws.com/sagemaker-us-east-1-537534971119/frozen-lake.tar.gz > frozen-lake.tar.gz
tar xvzf frozen-lake.tar.gz -C ~/SageMaker/frozen-lake/


## 安装冰雪奇缘湖游戏环境以及依赖

In [ ]:
!pip install -e ~/SageMaker/frozen-lake

安装完毕后需要重启kernel生效，
引入所需依赖

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import torch
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    HyperparameterTuner,
    ContinuousParameter,
    IntegerParameter,
)

from frozen_lake import (
    Level, LeveledFrozenLake,
    play_level, train, DeepQConfig,
    get_state, DeepQNetwork, moving_average,
    play_manually, get_test_level
)


## 运行游戏，先玩一下

使用OpenAI Gym 环境引入FrozenLakeEnv环境用于测试游戏。

openAI Gym
https://github.com/openai/gym
Gym 是一个开源 Python 库，通过提供用于在学习算法和环境之间进行通信的标准 API 以及一组符合该 API 的标准环境，用于开发和比较强化学习算法。自发布以来，Gym 的 API 已成为这样做的现场标准。

### 初始化游戏环境

In [ ]:
#https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
env = FrozenLakeEnv()
env.render()

左上角的“S”代表“开始”，右下角的“G”代表“目标”。 “F”和“H”分别代表“冻结”和“洞”。游戏的想法是导航到目标而不会掉入冰洞。这将是微不足道的，除非冰很滑，使您的动作不确定。

与环境交互的最重要的概念是状态、动作和奖励。只要板子被认为是固定的（稍后您将使用动态板子），状态只是光标的位置。环境对象将其存储在 s 属性中。

In [ ]:
env.s

当您向右和向下移动时，状态会增加。

动作是代理对下一步做什么的决定。在这个游戏中，有四种可能的动作：向左、向下、向右和向上。这四个动作分别用整数 0、1、2 和 3 表示。

环境的最后一个重要概念是奖励。在这个游戏中，达到目标的奖励为 1.0，所有其他步骤的奖励为 0.0。在训练期间，代理的目标是找到一个最大化奖励的策略。

In [ ]:
env = FrozenLakeEnv(is_slippery=False)
env.render()

关闭滑动模式并采取一些确定性操作的步骤来感受 API。在一个新单元格中，创建一个关闭滑动模式的新环境并渲染它：

In [ ]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

env.step(DOWN)
env.render()

为方向定义常量，使用 step() 方法执行操作并重新渲染板：

In [ ]:
env.step(RIGHT)
env.render()

### 使用API玩一下游戏

In [ ]:
env.reset()
env.step(DOWN)
env.step(DOWN)
env.step(RIGHT)
env.step(RIGHT)
env.step(DOWN)
env.step(RIGHT)
env.render()

现在您已经了解了环境的运作方式，最好手动玩几次游戏以了解游戏的运作方式。虽然目标是训练智能体自动玩游戏，但对游戏有一些直觉会帮助你制定有效的训练策略。尽管在关闭滑行模式时策略很明显，但对代理行为的非确定性响应改变了方法。

In [ ]:
## 注意需要使用notebook 才可以显示wiget，使用jupuyter notebook lab 按钮显示不了

### 通过图形化界面玩一下游戏

禁止冰块滑倒模式

In [ ]:
env = FrozenLakeEnv(is_slippery=false)
play_manually(env)

启用冰块滑倒模式

In [ ]:
env = FrozenLakeEnv(is_slippery=True)
play_manually(env)

到目前为止，已经有两种游戏模式：滑和不滑。滑倒模式非常困难而且违反直觉，优势后莫名其妙的改变指令的方向。这样的滑倒模式意义不大。默认情况下，你可以用来改变《冰雪奇缘湖》游戏难度的唯一办法是创建不同的棋盘。但是为了创建多个关卡，最好也能够调整犯错的概率。

In [ ]:
env.reset()

* 使用随机难度，增加犯错的概率

In [ ]:
random_level = LeveledFrozenLake.random(0.2)
play_manually(random_level)

这些关卡比非滑模式关卡难得多，但比 OpenAI Gym 中的默认游戏要容易得多（后者犯错的概率为 67％）。

## 训练一个机器人

### 随机代理（未经训练的机器人）

现在，你应该创建一个可以自动播放关卡的随机代理。这将作为您可以期望从学习的代理获得的性能的基准 — 学习的代理应该优于随机代理。

frozen-lake 软件包中的 play_level () 函数是一个帮助程序，允许代理玩《冰雪奇缘湖》游戏的某个关卡

* 该函数有两个参数：env 和 get_action ()。env 参数是冰冻湖环境。另一个参数是可调用对象 —— 一个定义游戏行为的函数。get_action () 可调用对象将接受一个环境并返回一个操作。

* play_level () 函数重置环境，然后执行操作，直到游戏完成 或 代理完成 100 个步骤（以先到者为准）。然后它返回的奖励将是 0.0 或 1.0，具体取决于是否赢了该等级。

* 要评估随机代理，你需要一个测试级别和一个 get_action () 可调用对象，它在每个步骤都返回一个随机动作。

构建随机代理进行测试
定义随机行为

In [ ]:
def random_action(env: FrozenLakeEnv) -> int:
    """Choose a random action"""
    return np.random.randint(env.nA)

In [ ]:
play_level(get_test_level(), random_action)

使用随机代理，玩10000次游戏

In [ ]:
# Run a random agent on the same level 10,000 times
np.random.seed(1)
n_attempts = 10000
test_level = get_test_level()
rewards = [
    play_level(test_level, random_action)
    for _ in range(n_attempts)
]

现在计算随机代理的获胜百分比：

In [ ]:
sum(rewards) / n_attempts

In [ ]:
sum(rewards)

## 训练一个有智慧的代理（机器人）
为了训练这个机器人我们需要将当前的状态信息向量化，这里通过一个矩阵来表示，每一个cell表示当前的状态 0表示冻结，1表示洞

请记住，Frozen Lake 环境的状态是 [0, 15] 范围内的整数，表示光标在 4x4 板上的位置。如果看板是固定的，这很有效，因为要对下一个操作做出正确的决定，你唯一需要的信息就是当前位置。

* 但是，如果同一个代理要玩多个不同的棋盘，那么代理需要有关注两部分信息：板的配置和光标的位置。

状态信息可以很容易通过数组或更高维度的张量。为了简单起见，在本教程中，state通过一维数组表示。

* 这样board将永远是 4x4的矩阵。你还知道左上角（位置 0）始终是 'S' 表示开始，右下角（位置 15）始终是 'G' 表示目标。因为这两个单元格是固定的，所以你不需要把这些单元格放在状态数组中。
* 还有 14 个单元，它们可以是 “F” 表示冻结，也可以是 “H” 表示洞。由于状态数组需要是数字，因此使用 0 表示冻结，使用 1 表示从左向右移动的 H（按行主要顺序）。

使用两个 one-hot 向量来存储光标的位置：一个用于光标行，另一个用于列。要将单个整数位置转换为两个 one-hot 向量，请获取整数位置的行 (m) 和列 (n)。一个one-hot向量将是 4x4 单位矩阵的第 n 行和第 n 行。

在 frozen-lake 软件包中有一个名为 get_state () 的函数，它在给定状态下为游戏板生成一个完整的状态数组。看看 frozen_lake/state.py 看看它是如何工作的。

In [ ]:
test_level = get_test_level()
test_level.render()
get_state(test_level)

### 本地训练一个机器人（策略网络）

开始新游戏
对于游戏中的每一步：
* 通过策略网络运行游戏状态以获取操作
* 执行操作并获取新状态
* 使用当前奖励（或缺少奖励）更新保单网络
* 游戏完成后，打破循环
* 训练运行的剧集数量可配置。

策略神经网络由两个线性层组成，在第一层之后激活一个 relU，在第二层之后激活一个 softmax。softmax 层输出一个大小为 4 的数组（四个操作中的每个操作一个元素）。最佳操作应与最高输出相对应。但是，在训练和模拟游戏过程中，最好引入一些随机性以防止代理卡住。

策略网络是在 frozen_lake/model.py 中定义的 —— 看看它是如何工作的。当然，有可能使网络更深入、更复杂。但是对于简单的游戏来说，保持网络规模较小是个好主意，因为游戏很简单，小型网络的训练速度更快。

如果您按照上述的高级程序训练此网络，则很难学习如何很好地发挥冰冻湖的作用的策略。强化学习往往不稳定，并且有一些修改对于它在实践中很好地发挥作用非常重要：体验重放和目标网络。

在实际更新网络之前，体验重放会跟踪几个步骤的奖励。目标网络是策略网络的克隆，更新频率较低。

这两个修改的实现都基于 PyTorch 站点的教程。查看 frozen_lake/memory.py 和目标网络中体验重播的内存，并更新 frozen_lake/train.py 中的步进逻辑。

####  使用DeepQ网络训练
* DeepQ网络介绍：https://towardsdatascience.com/beating-video-games-with-deep-q-networks-7f73320b9592
* Pytorch深度学习介绍：https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [ ]:
config = DeepQConfig()

In [ ]:
local_policy, reward = train(config)

In [ ]:
plt.plot(moving_average(reward))
plt.ylabel('reward')
plt.xlabel('episode');


你可以看到，即使进行了体验重播和目标网络修改，奖励仍然有些不稳定。这是使用超参数搜索扩大训练范围以便找到效果最佳的策略的一个很好的理由。

现在尝试之前的标准测试级别上的策略网络并打印结果：

In [ ]:
np.random.seed(1)
n_attempts = 10000
test_level = get_test_level()
rewards = [
    play_level(test_level, local_policy.learned_action)
    for _ in range(n_attempts)
]
sum(rewards) / n_attempts


获胜百分比应高于之前的随机代理。

## 使用Sagemaker进行训练

由于学习策略的性能不稳定 —— 不足以估计人类的表现，因此最好将训练扩展到跨不同超参数的多次尝试。SageMaker 让这变得非常简单。首先，您将向单个训练作业发送训练脚本，然后使用超参数调整功能搜索超参数值。

要在 SageMaker 上训练作业，您需要一个脚本作为训练容器的入口点。SageMaker 会将超参数作为命令行参数传入。因此，例如，如果你的脚本名为 main.py，并且你想要传入 target_update 值 5 作为超参数，SageMaker 将在训练容器中运行命令 python main.py —target_update 5。

脚本需要训练模型并将其保存到特定位置的磁盘中。然后，SageMaker 会将模型上传到 S3，在那里它可以在本地或托管终端节点中使用。

在本教程中，main.py 脚本已经编写完成，它位于带有 setup.py 的顶级冷冻湖文件夹中。要在 SageMaker 上将其作为训练作业运行：

集成训练代码到Sagemaker使用BYOS

In [ ]:
estimator = PyTorch(
    entry_point='main.py',
    source_dir='/home/ec2-user/SageMaker/frozen-lake/',
    framework_version='1.2.0',
    train_instance_type='ml.m5.large',
    train_instance_count=1,
    role=get_execution_role(),
    py_version='py3'
)

In [ ]:
estimator.fit()

下载模型并解压
什么是 PTH 文件？PTH 文件大多属于 PyTorch。PTH 是使用 PyTorch 进行机器学习的数据文件。PyTorch 是一个基于 Torch 库的开源机器学习库。它主要由 Facebook 人工智能研究小组开发。如何打开 PTH 文件。你需要像 PyTorch 这样的合适软件来打开

In [ ]:
!aws s3 cp $estimator.model_data ./
!tar xvzf model.tar.gz
!rm model.tar.gz

策略网络权重现在将位于。/policy.pth。

在新单元格中，创建一个新的策略网络并将权重加载到内存中：

In [ ]:
sagemaker_policy = DeepQNetwork(config.n_state_features, config.n_actions)
sagemaker_policy.load_state_dict(torch.load('policy.pth'))

在测试级别评估性能：

In [ ]:
np.random.seed(1)
n_attempts = 10000
test_level = get_test_level()
rewards = [
    play_level(test_level, sagemaker_policy.learned_action)
    for _ in range(n_attempts)
]
sum(rewards) / n_attempts


性能应该与 local_policy 网络相同，因为所有随机种子都是固定的。

## 超参数优化
大约需要20分钟左右

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name='MaxReward',
    metric_definitions=[
        dict(
            Name='MaxReward',
            Regex='MaxReward=([0-9\\.]+)',
        )
    ],
    hyperparameter_ranges=dict(
        target_update=IntegerParameter(10, 500),
        epsilon_start=ContinuousParameter(0.25, 0.75),
    ),
    max_jobs=20,
    max_parallel_jobs=5,
)

In [ ]:
tuner.fit()
tuner.wait()

In [ ]:
model_path = (
    estimator.output_path +
    tuner.best_training_job() +
    '/output/model.tar.gz'
)

!aws s3 cp $model_path ./
!tar xvzf model.tar.gz
!rm model.tar.gz


In [ ]:
tuned_policy = DeepQNetwork(config.n_state_features, config.n_actions)
tuned_policy.load_state_dict(torch.load('policy.pth'))

n_attempts = 10000
rewards = [
    play_level(test_level, tuned_policy.learned_action)## 原workshop有误，参数应该为test_level，文档中是test_env 
    for _ in range(n_attempts)
]
sum(rewards) / n_attempts

## 按难度排序关卡

创建十个新关卡，并在每个级别上运行学习过的代理 10,000 次。然后存储难度（即 1-win_百分比）、出错的概率和冰洞的数量。按难度对关卡进行排序：

In [ ]:
np.random.seed(1)
levels = []
for i in range(10):
    print(i)
    level_config = Level.random(config.p_mistake_draw)
    level = LeveledFrozenLake(level_config)
    win_precentage = sum(
        play_level(level, tuned_policy.learned_action)
        for _ in range(n_attempts)
    ) / n_attempts
    n_holes = (np.array(list(''.join(level_config.board))) == 'H').sum()
    levels.append(dict(
        difficulty=1-win_precentage,
        p_mistake=level_config.p_mistake,
        n_holes=n_holes,
        level=level,
    ))

levels = sorted(levels, key=lambda l: l['difficulty'])

这将需要大约十分钟的时间才能运行，因为代理程序必须为每个步骤调用策略网络。

In [ ]:
level_df = pd.DataFrame(levels)
level_df = level_df.sort_values('difficulty')## 难度= （1-对应的就是通关率【win_precentage 】）
level_df[['difficulty', 'p_mistake', 'n_holes']]

难度= （1-对应的就是通关率【win_precentage 】） 也就是说对应级别的难度很低，通关率很高